In [ ]:
MySQL Physical Cold Backup
=============================================

-- Cold backup involves physically copying database related files to backup device
   taking the database down.
-- for this the mysql database server should be clean shutdown.

files required in cold backup
==========================================

-- all the files in datadir.
-- all the files of system tablespaces.
-- all the configuration/option files.

files not required in cold backup
============================================

-- redo log files , undo log files , bin log files 
-- temp files , double write buffer files.


clean shutdown the database
-----------------------------------------

mysql> set global innodb_fast_shutdown = 0;

[root@mysqlserver mysql]# systemctl stop mysqld


copy files to backup location 
------------------------------------------

[root@mysqlserver mysql]# cp -r /var/lib/mysql/data /tmp/mysql_cold_backup/

[root@mysqlserver mysql]# cp -r /var/lib/mysql/systemdata /tmp/mysql_cold_backup/

[root@mysqlserver mysql]# cp -p /etc/mysql/my.cnf /tmp/mysql_cold_backup/

[root@mysqlserver mysql]# cp -p /var/lib/mysql-files/*.cnf /tmp/mysql_cold_backup/


to restore mysql instance from the cold backup , remove all the files 
-------------------------------------------------------------------------

[root@mysqlserver mysql]# systemctl stop mysqld
[root@mysqlserver mysql]# 

[root@mysqlserver mysql]# rm -rf /var/lib/mysql/tmpdir/*
[root@mysqlserver mysql]# rm -rf /var/lib/mysql/doublewrite/*
[root@mysqlserver mysql]# rm -rf /var/lib/mysql/redologs/*
[root@mysqlserver mysql]# rm -rf /var/lib/mysql/undolog/*
[root@mysqlserver mysql]# rm -rf /var/lib/mysql/tempdata/*
[root@mysqlserver mysql]# rm -rf /var/lib/mysql/data/*
[root@mysqlserver mysql]# rm -rf /var/lib/mysql/binlog/*
[root@mysqlserver mysql]# rm -rf /var/lib/mysql/systemdata/*


copy the files from backup
-------------------------------------

[root@mysqlserver mysql_cold_backup]# cp -r /tmp/mysql_cold_backup/data/* /var/lib/mysql/data/
[root@mysqlserver mysql_cold_backup]# cp -r /tmp/mysql_cold_backup/systemdata/* /var/lib/mysql/systemdata/

[root@mysqlserver mysql_cold_backup]# chown -R mysql:mysql /var/lib/mysql/data/
[root@mysqlserver mysql_cold_backup]# chown -R mysql:mysql /var/lib/mysql/systemdata/


start the mysql instance 
-------------------------------------

[root@mysqlserver mysql]# systemctl start mysqld
[root@mysqlserver mysql]# 

In [ ]:
Logical Backups
==============================

-- does not involve copying of physical files.
-- all databases , some databases and table level backup can be taken.
-- utility have to be used like mysqldump amd mysqlpump.
-- takes backup in .sql format , text delimited or XML format.

[mysqluser@mysqlserver backups]$ mysqldump --verbose --help

Dumping structure and contents of MySQL databases and tables.
Usage: mysqldump [OPTIONS] database [tables]
OR     mysqldump [OPTIONS] --databases [OPTIONS] DB1 [DB2 DB3...]
OR     mysqldump [OPTIONS] --all-databases [OPTIONS]


specifying defaults for mysqldump 
--------------------------------------------

[root@MySqlServer1 backups]# cat /etc/my.cnf
[mysqldump]
host=localhost
user=db_admin
password=Dbadmin@2024

[root@MySqlServer1 backups]# mysqldump --print-defaults

mysqldump would have been started with the following arguments:
--host=localhost --user=db_admin --password=***** 
[root@MySqlServer1 backups]# 


Single Table Backup
-----------------------

mysqldump sakila actor > sakilaDB_actorTable_bkp.sql


Whole Single Database Backup 
-------------------------------

mysqldump sakila > sakilaDB_bkp.sql


Two databases backup
--------------------------

mysqldump --databases posdb sakila > posdb_sakila_bkp.sql


All databases backup 
-----------------------------

mysqldump --all-databases > all_dbs_bkp.sql


Database backup with skipping some tables
-------------------------------------------------

mysqldump sakila --ignore-table=sakila.actor --ignore-table=sakila.film_actor > 
          sakiladb_skipactor_bkp.sql


Multiple database backup with skipping some tables
-------------------------------------------------------

mysqldump --databases sakila posdb --ignore-table=sakila.actor --ignore-table=sakila.film_actor 
          --ignore-table=posdb.emp_salaries > sakiladb_posdb_ignoretables_bkp.sql


consistent backup 
-------------------------------------------------

mysqldump --single-transaction > all_dbs_consistent_bkp.sql


consistent backup without locking tables
-------------------------------------------------

mysqldump --all-databases --single-transaction --skip-lock-tables > all_dbs_consistent_bkp_dump.sql


definition backup without data 
--------------------------------------------

mysqldump -u root -p --no-data sakila > /home/mysqluser/backups/sakila_db_backup_definition.sql


only data backup without definition ( create statements )
------------------------------------------------------------------

mysqldump -u root -p --no-create-info sakila > /home/mysqluser/backups/sakila_db_backup_dataonly.sql


backup databases with compressed backup file 
------------------------------------------------------

mysqldump -u root -p --databases sample sakila | gzip > dump_db_sample_sakila.sql.gz


Dump large tables with quick option 
-----------------------------------------------

mysqldump --comments --quick airportdb > airportdb_dump_quick.sql


copy data from one host to another using mysqldump
-------------------------------------------------------

mysqldump ordersdb | mysql -h 192.168.153.137 -u db_admin -p******** orders


Dump a remote database
--------------------------------------------

mysqldump -h bosprod01 --port 3306 -u db_admin -p sakila > dump_bosprod01_sakiladb.sql


Use where clause to filter data
-----------------------------------------

mysqldump sample sample_employees --where="employee_id > 1000" > dump_sample_employees_custid1000.sql


Limit backup file using where clause
------------------------------------------------

mysqldump sample --where='1 limit 500' > dump_sample_500records.sql


Dump database with routines ( by default mysqldump does not include routines)
---------------------------------------------------------------------------------

mysqldump Northwind --routines > dump_northwind.sql




Single Table Restore from backup taken for single table
---------------------------------------------------------------

mysql> drop table actor;

mysql -u db_admin -p sakila < sakilaDB_actorTable_bkp.sql


Single Table restore from backup taken for full database
--------------------------------------------------------------

ALL CREATE TABLE statements start with DROP TABLE IF EXISTS and then CREATE TABLE and then
INSERT STATEMENTS and it will end with keyword UNLOCK TABLES

sed -n -e '/DROP TABLE.*`film_actor`/,/UNLOCK TABLES/p' sakilaDB_bkp.sql > sakilaDB_filmactor_bkp.sql

drop table actor ;

mysql sakila < sakilaDB_filmactor_bkp.sql


Database restore from backup taken for a single database
---------------------------------------------------------------------

mysql> drop database sakila;

mysql sakila < sakilaDB_bkp.sql


Restore a single database from all database backup
-------------------------------------------------------

cat dump_alldatabases.sql | grep 'CREATE DATABASE' | grep -v mysql

mysql> drop database ordersdb;

mysql> drop database sakila;

If the create database is in between the dump 
----------------------------------------------------------

sed -n -e '/CREATE DATABASE.*`ordersdb`/,/CREATE DATABASE/p' dump_alldatabases.sql > ordersdb_bkp.sql

remove the last create datbase statement in ordersdb_bkp.sql

mysql < ordersdb_bkp.sql

If the create database is the last 
----------------------------------------------------------

sed -n -e '/CREATE DATABASE.*`sakila`/,/Dump completed/p' dump_alldatabases.sql > sakiladb_bkp.sql

mysql < sakiladb_bkp.sql

In [ ]:
MySql Hot Backup with Xtrabackup 
====================================================

-- Hot backup involves physically copying database related files to backup device
   when the database is Up.

-- Tools - mysqlbackup ( oracle , paid) , mariadbbackup ( mariadb , open-source)
           xtrabackup ( percona , open-source )

-- point-in-time recovery can be performed using hot backup and binary logs.


download xtrabackup utility
----------------------------------------------

https://www.percona.com/downloads#percona-xtrabackup

yum localinstall percona-xtrabackup-80-8.0.30-23.1.el8.x86_64.rpm 

xtrabackup --version

create user for taking backups and grant permissions
---------------------------------------------------------

mysql> create user 'bkpuser'@'%' identified by 'Bkpuser@2024';

mysql> grant reload , lock tables , process , replication client on *.* to 'bkpuser'@'%' ;

mysql> grant backup_admin on *.* to 'bkpuser'@'%' ;

mysql> grant select on performance_schema.log_status to 'bkpuser'@'%' ;

mysql> grant select on performance_schema.keyring_component_status to 'bkpuser'@'%' ;

mysql> flush privileges;


Take full backup of mysql instance with xtrabackup 
---------------------------------------------------------

xtrabackup --backup --target-dir=/home/mysqluser/backups

xtrabackup --prepare --target-dir=/home/mysqluser/backups


Restore using the backup taken through xtrabackup utility
--------------------------------------------------------------

systemctl stop mysqld

rm -rf /var/lib/mysql/*  ( remove all files except binlog and errorlog )

xtrabackup --copy-back --target-dir=/home/mysqluser/backups/

chown -R mysql:mysql /var/lib/mysql/

systemctl start mysqld




xtrabackup incremental backups
-------------------------------------------

create a full base backup of the database
-------------------------------------------------------

xtrabackup --backup --target-dir=/home/mysqluser/backups/base

create an incremental-1 database backup 
-------------------------------------------

xtrabackup --backup --target-dir=/home/mysqluser/backups/incr1 
           --incremental-basedir=/home/mysqluser/backups/base

create an incremental-2 database backup
----------------------------------------------

xtrabackup --backup --target-dir=/home/mysqluser/backups/incr2 
           --incremental-basedir=/home/mysqluser/backups/incr1


prepare the full and incremental backups for restoration
----------------------------------------------------------------

Need to add --apply-log-only option when preparing incremental backups except for the last one.
This is to prevent the rollback phase , else the incremental backups becomes useless.

xtrabackup --prepare --apply-log-only --target-dir=/home/mysqluser/backups/base

xtrabackup --prepare --apply-log-only --target-dir=/home/mysqluser/backups/base 
           --incremental-dir=/home/mysqluser/backups/incr1

xtrabackup --prepare --target-dir=/home/mysqluser/backups/base 
           --incremental-dir=/home/mysqluser/backups/incr2




Point-In-Time Recovery using Xtrabackup and binary logs
----------------------------------------------------------------

run the full backup 
----------------------------

xtrabackup --backup --target-dir=/home/mysqluser/backups

xtrabackup --prepare --target-dir=/home/mysqluser/backups

prepare for restoration by emptying all the mysql data , systemdata , redolog directories
---------------------------------------------------------------------------------------------

Don't remove files in errorlog and binlog directories.

systemctl stop mysqld

[root@mysqlserver backups]# rm -rf /var/lib/mysql/data/*
[root@mysqlserver backups]# rm -rf /var/lib/mysql/tempdata/*
[root@mysqlserver backups]# rm -rf /var/lib/mysql/systemdata/*


move binlogs to different location , if there is backup of binlog you can remove this
----------------------------------------------------------------------------------------

mv posb_binlog.* ../old_binlog/


restore all the files from the full backup and verify
--------------------------------------------------------------------------

xtrabackup --copy-back --target-dir=/home/mysqluser/backups

chown -R mysql:mysql /var/lib/mysql/


check the xtrabackup_binlog_info for log file name and position
---------------------------------------------------------------------------

[root@mysqlserver backups]# cat xtrabackup_binlog_info
posb_binlog.000010	493
[root@mysqlserver backups]# 


create a restore file with this info and the time to which the database has to be restored 
--------------------------------------------------------------------------------------------

mysqlbinlog /var/lib/mysql/old_binlog/posb_binlog.000010 --start-position=493 
            --stop-datetime="2024-05-21 18:02:30" > restore_db.sql


start the mysqld server and run the restore_db.sql
----------------------------------------------------------

systemctl start mysqld

mysql < restore_db.sql



xtrabackup compressed backup 
---------------------------------------

xtrabackup --backup --compress --target-dir=/home/mysqluser/backups/full_compressed


compressed backups should be decompressed before prepare and restore
------------------------------------------------------------------------

xtrabackup --decompress --target-dir=/home/mysqluser/backups/full_compressed



Creating partial backups
---------------------------------------------

all tables in a database 
--------------------------------

xtrabackup --backup --target-dir=/home/backups/ordersdb_all_tables/ --tables="^ordersdb[.].*"


single table in a database 
----------------------------------

xtrabackup --backup --target-dir=/home/backups/ordersdb_products01_table/ 
           --tables="^ordersdb[.]products01"


table files option 
-----------------------

cat ordersdb_tables.txt
ordersdb.products01
ordersdb.products02

xtrabackup --backup --target-dir=/home/backups/ordersdb_tables/ 
           --tables-file=/home/mysqluser/backups/ordersdb_tables.txt


single database backup
--------------------------------------

xtrabackup --backup --databases='mysql sys performance_schema ordersdb' 
           --target-dir=/home/mysqluser/backups/ordersdb_bkp/


database files option
-----------------------------------------

cat databases_for_bkp.txt 
mysql
sys
performance_schema
salesdb

xtrabackup --backup --databases-file=/home/mysqluser/backups/databases_for_bkp.txt
           --target-dir=/home/mysqluser/backups/databases_bkp/


preparing partial backups and restore 
----------------------------------------------------------

xtrabackup --prepare --export --target-dir=/home/mysqluser/backups/ordersdb_all_tables

if table is present and you want to restore 
---------------------------------------------

mysql> alter table ordersdb.products01 discard tablespace;

cp /home/mysqluser/backups/ordersdb_all_tables/ordersdb/products01.*  /var/lib/mysql/data/ordersdb/

chown mysql:mysql /var/lib/mysql/data/ordersdb/products01.*

mysql> alter table ordersdb.products01 import tablespace;

if table is dropped and you want to restore
---------------------------------------------------

CREATE TABLE `products02` (
  `prod_id` int NOT NULL,
  `prod_details` varchar(100) DEFAULT NULL,
  `prod_date` datetime DEFAULT NULL,
  PRIMARY KEY (`prod_id`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_0900_ai_ci;

alter table products02 discard tablespace;

cp /home/mysqluser/backups/ordersdb_all_tables/ordersdb/products02.* 
   /var/lib/mysql/data/ordersdb/

chown mysql:mysql /var/lib/mysql/data/ordersdb/products02.*

alter table products02 import tablespace;

In [ ]:
MySql Replication 
=======================================

-- Replication is a copy of mysql server which is always in sync.
-- source server is called master/primary and the destination server is called slave/replica.
-- mysql replication is asyncrhonus by default.
-- mysql replication can be of all databases , some databases or even some tables.

Two methods of mysql replication 
======================================

-- traditional binary log file and position based replication.
-- GTID Global Transaction Ids based replication

Replication format
================================

-- Statement based replication.
-- Row based replication.
-- mixed based replication.


Statement based replication 
=====================================

-- binlog_format = Statement , sql statements are transferred and executed in replica.
-- consumes less space .
-- non-deterministic objects are difficult to replicate.

Row based replication
======================================

-- binlog_format = row , chnaged rows are transferred and applied in replica.
-- consumes more space .
-- cannot see what is transferred and applied in replica.

Mixed based replication 
========================================

-- binlog_format = mixed , provides best combination of data integrity and performance.
-- recommended format in most cases.



Primary Server Setup 
================================

create a seperate option file replcation.cnf
-------------------------------------------------------
[mysqld]
log-bin = /var/lib/mysql/binlog/posb_binlog
log-bin-index = /var/lib/mysql/binlog/posb_binlog.index
binlog-format = MIXED
server-id = 1

create a dedicated replication user 
-------------------------------------------------------
mysql> create user replicator identified by 'P@ssword2024';
mysql> grant replication slave on *.* to replicator;

backup and prepare in primary 
-------------------------------------
xtrabackup --backup --target-dir=/home/mysqluser/backups/bkp_for_replication
xtrabackup --prepare --target-dir=/home/mysqluser/backups/bkp_for_replication

copy backup files from primary to replica server 
-----------------------------------------------------
scp -r bkp_for_replication root@192.168.153.134:/home/mysqluser/backups/

stop the replica db server 
-----------------------------------------------
systemctl stop mysqld

remove all the data , redo , undo , systemdata files in replica server
----------------------------------------------------------------------------
pwd
/var/lib/mysql
rm -rf *

copy all the files from the backup location to mysql datadir location
-------------------------------------------------------------------------
cp -r /home/mysqluser/backups/bkp_for_replication/* /var/lib/mysql/
chown -R mysql:mysql /var/lib/mysql/

start the mysqld server 
----------------------------
systemctl start mysqld


Replica server setup 
===========================================

create a seperate option file replcation.cnf
----------------------------------------------
[mysqld]
relay-log = /var/lib/mysql/relaylog/posb_relaylog
relay-log-index = /var/lib/mysql/relaylog/posb_relaylog.index
server-id = 2
skip-replica-start
read-only

pwd
/var/lib/mysql
mkdir relaylog
chown -R mysql:mysql relaylog/

systemctl restart mysqld

Get binary log file name and position from xtrabackup 
-------------------------------------------------------------
cat xtrabackup_binlog_info
posb_binlog.000029	157


Setup replica in replica server 
-------------------------------------------
mysql> show replica status;
mysql> change master to
    -> master_host = '192.168.153.133',
    -> master_user = 'replicator' ,
    -> master_password = 'P@ssword2024' ,
    -> master_log_file = 'posb_binlog.000029' ,
    -> master_log_pos = 157;
mysql> start slave;
mysql> show slave status \G


Stopping IO thread and SQL thread 
=====================================================

stop IO thread in slave 
-----------------------------------------
mysql> show slave status \G
mysql> stop slave IO_THREAD;
mysql> show slave status \G

mysql> start slave IO_THREAD;


change options in replica server to auto start slave when mysqld restart
============================================================================

skip-replica-start = false

systemctl restart mysqld

mysql> show slave status \G


Replicate from a source database to different replica database
==========================================================================

In replica / slave
----------------------------------------------------------

vi /etc/replication.cnf 

replicate-rewrite-db = financedb->financedbreplica


Replicate only a single database 
====================================================


In replica / slave
----------------------------------------------------------

vi /etc/replication.cnf 

replicate-do-db = salesdb



Replicte 2 databases 
==============================================

replicate-do-db = salesdb
replicate-do-db = ordersdb


Replicate a single table 
==============================================

replicate-do-table = ordersdb.products01


Replicate with wild cards
=============================

replicate-wild-do-table = ordersdb.prod%
replicate-wild-do-table = order%.prod%


Replicate with ignore 
=============================

Replicate-Ignore-DB = financedb
Replicate-Ignore-Table = financedb.emp_salary
Replicate-Wild-Ignore-Table = finance%.%salary%


Reset slave and make it stand-alone
============================================================

mysql> show master status;
mysql> show slave status \G
mysql> reset slave all;
mysql> show slave status \G

remove all options related to replica 
----------------------------------------------
systemctl stop mysqld
vi /etc/replication.cnf 
systemctl start mysqld
reset master;


In [ ]:
GTID ( Global Transaction Identifier )
====================================================

-- is a unique identifier created and associated with each transaction committed in the source server.
-- if a transaction is not written to the binary log , it is not assigned a GTID.
-- replicated transactions retain the same GTID that was assigned to it on the source.
-- the mysql.gtid_executed table is used to preserve the GTIDs.
-- GTID is represented by pair of characters seperated by colon -> source_id : transaction_id
-- cde07bc2-178f-11ef-b846-000c29a50cb9:1-4
-- RESET MASTER will clear the GTIDs and clear all the data in mysql.gtid_executed table.

GTID Auto-Positioning 
============================

-- for gtid based replication you have to enable source_auto_position which is disabled by default.
-- in the initial handshake the replica sends a gtid set that it has already received.
-- the source responds by sending all transactions which is not included in the gtid set sent by replica.

In [ ]:
MySql replication with GTID 
===========================================

Start both servers with GTID enabled 
----------------------------------------------

In primary server 
---------------------------
 
cat /etc/replication.cnf
[mysqld]
server-id = 1
log-bin = /var/lib/mysql/binlog/mysqlprod01_binlog
log-bin-index = /var/lib/mysql/binlog/mysqlprod01_binlog.index
binlog-format = MIXED
gtid-mode = ON
enforce-gtid-consistency = ON


In replica server
---------------------------------

cat /etc/replication.cnf 
[mysqld]
server-id = 2
log-bin = /var/lib/mysql/mysqlprod02_binlog
log-bin-index = /var/lib/mysql/mysqlprod02_binlog.index
relay-log = /var/lib/mysql/relaylog/mysqlprod02_relaylog
relay-log-index = /var/lib/mysql/relaylog/mysqlprod02_relaylog.index
binlog-format = MIXED
skip-replica-start
gtid-mode = ON
enforce-gtid-consistency = ON


stop and start both servers
------------------------------------

systemctl stop mysqld
systemctl start mysqld


create a dedicated replication user in primary
-----------------------------------------------------------------------------------------

create user replicator identified by 'P@ssword2024';
grant replication slave on *.* to replicator;

Configure the replica to use GTID-based auto positioning 
-----------------------------------------------------------------

mysql> change replication source to
    -> source_host = '192.168.153.133',
    -> source_port = 3306,
    -> source_user = 'replicator',
    -> source_password = 'P@ssword2024',
    -> source_auto_position = 1;

mysql> start replica;

mysql> show replica status \G


Add one more replica to the replication environment from the bakup of first replica
-------------------------------------------------------------------------------------------

remove data directory files from new replica server 
--------------------------------------------------------------------
systemctl stop mysqld
rm -rf /var/lib/mysql/*

stop the existing replica and copy the data directory including log files and relay log files
into the new replica server.
------------------------------------------------------------------------------------------------------
mysql> stop replica;
systemctl stop mysqld
scp -r /var/lib/mysql/* root@bosprod03:/var/lib/mysql/

after copying , delete the auto.cnf in new replica server so that a new 
server uuid is created in the new server.
-------------------------------------------------------------------------------------------
chown -R mysql:mysql /var/lib/mysql
rm -rf /var/lib/mysql/auto.cnf

edit the config file in new replica server to add replication options 
------------------------------------------------------------------------------
cat /etc/my.cnf
[mysqld]
server-id = 3

restart the old replica 
----------------------------------------------
systemctl start mysqld
mysql> start replica;
mysql> show slave status \G

start the new replica 
----------------------------------------------

systemctl start mysqld
mysql> start replica;


Make the replicas read only
----------------------------------------------

mysql> stop replica;
systemctl stop mysqld
cat /etc/my.cnf
read-only = TRUE
systemctl start mysqld
mysql> show variables like '%read_only%';
mysql> start replica;


Views and tables to check replication
==================================================================

in primary 
------------------------

mysql> select * from mysql.gtid_executed;

mysql> show master status;

mysql> show replicas;

mysql> select * from performance_schema.processlist  where user = 'replicator';


in replica 
---------------------------------------------------------

mysql> show replica status \G

mysql> select * from mysql.slave_master_info;

mysql> select * from performance_schema.replication_connection_status \G

select * from performance_schema.replication_connection_configuration \G

select * from performance_schema.replication_applier_status;

select channel_name,thread_id,service_state 
from performance_schema.replication_applier_status_by_coordinator;

select channel_name,worker_id,thread_id,service_state from 
performance_schema.replication_applier_status_by_worker;


Make the replicas read_only
-----------------------------------------------------

mysql> stop replica;

[root@bosprod02 ~]# systemctl stop mysqld

[root@bosprod02 ~]# cat /etc/my.cnf
read-only = TRUE

[root@bosprod02 ~]# systemctl start mysqld

mysql> show variables like '%read_only%';


Switching Sources during failover 
-----------------------------------------------------

suppose the soure/primary server ( bosprod01 ) is down. Make bosprod02 primary and link replica bosprod03 to 
new primary bosprod02.

mysql> select @@hostname;
| bosprod02  |

mysql> stop replica;

mysql> reset slave all;

mysql> show replica status \G
Empty set (0.00 sec)

remove read-only , relay-log and skip-slave-start options in the config file.

[root@bosprod02 ~]# vi /etc/my.cnf
[root@bosprod02 ~]# systemctl restart mysqld

mysql> select @@hostname;
| bosprod03  |

mysql> stop replica;

mysql> change replication source to
    -> source_host = 'bosprod02',
    -> source_port = 3306,
    -> source_user = 'replicator',
    -> source_password = 'Repl@2024',
    -> source_auto_position = 1;

mysql> start replica;

mysql> show replica status \G


Once the old primary is up , we can make it a replica of new primary 

mysql> change replication source to
    -> source_host = 'bosprod02',
    -> source_port = 3306,
    -> source_user = 'replicator',
    -> source_password = 'Repl@2024',
    -> source_auto_position = 1;
Query OK, 0 rows affected, 2 warnings (0.08 sec)

mysql> start replica;

mysql> show replica status \G

In [ ]:
MySql Group Replication 
=========================================


Configuring an instance for group replication
---------------------------------------------------------------------

[root@bosprod01 ~]# cat /etc/my.cnf
[mysqld]

disabled_storage_engines="MyISAM,BLACKHOLE,FEDERATED,ARCHIVE,MEMORY"

server_id=1
gtid_mode=ON
enforce_gtid_consistency=ON

plugin_load_add='group_replication.so'
group_replication_group_name="5842bb1d-28ef-11ef-9a91-000c293ea0bc"
group_replication_start_on_boot=off
group_replication_local_address= "bosprod01:33061"
group_replication_group_seeds= "bosprod01:33061,bosprod02:33061,bosprod03:33061"
group_replication_bootstrap_group=off
[root@bosprod01 ~]# 

[root@bosprod01 ~]# systemctl restart mysqld


Create User For Distributed Recovery
-------------------------------------------------------------------

mysql> set sql_log_bin = 0;
mysql> create user replicator identified by 'Repl@2024';
mysql> grant replication slave on *.* to replicator;
mysql> grant connection_admin on *.* to replicator;
mysql> grant backup_admin on *.* to replicator;
mysql> grant group_replication_stream on *.* to replicator;
mysql> flush privileges;
mysql> set sql_log_bin = 1;


Launching Group Replication
-------------------------------------------------------------------------

mysql> CHANGE REPLICATION SOURCE TO SOURCE_USER='replicator' ,
    -> SOURCE_PASSWORD='Repl@2024'
    -> FOR CHANNEL 'group_replication_recovery';

mysql> show plugins;


Bootstrapping the Group
-------------------------------------------------------------

The process of starting a group for the first time is called bootstrapping.
The bootstrap should only be done by a single server, the one that starts the group and only once.

mysql> SET GLOBAL group_replication_bootstrap_group=ON;
mysql> START GROUP_REPLICATION;
mysql> SET GLOBAL group_replication_bootstrap_group=OFF;

mysql> select * from performance_schema.replication_group_members;


Adding a second instance to the group 
----------------------------------------------------------------------------

[root@bosprod02 ~]# cat /etc/my.cnf
[mysqld]

disabled_storage_engines="MyISAM,BLACKHOLE,FEDERATED,ARCHIVE,MEMORY"

server_id=2
gtid_mode=ON
enforce_gtid_consistency=ON

plugin_load_add='group_replication.so'
group_replication_group_name="5842bb1d-28ef-11ef-9a91-000c293ea0bc"
group_replication_start_on_boot=off
group_replication_local_address= "bosprod02:33061"
group_replication_group_seeds= "bosprod01:33061,bosprod02:33061,bosprod03:33061"
group_replication_bootstrap_group=off
[root@bosprod02 ~]# 

[root@bosprod02 ~]# systemctl restart mysqld


Create User For Distributed Recovery same as in instance1
-------------------------------------------------------------------


Start group replication
-------------------------------------------

mysql> CHANGE REPLICATION SOURCE TO SOURCE_USER='replicator' ,
    -> SOURCE_PASSWORD='Repl@2024'
    -> FOR CHANNEL 'group_replication_recovery';
Query OK, 0 rows affected, 2 warnings (0.09 sec)

mysql> START GROUP_REPLICATION;

mysql> select * from performance_schema.replication_group_members;



Restarting a Group 
-----------------------------------------------------

Group Replication is designed to ensure that the database service is continuously available, even if some of the servers 
that form the group are currently unable to participate in it due to planned maintenance or unplanned issues. 
As long as the remaining members are a majority of the group they can elect a new primary and continue to function as a group. 

However, if every member of a replication group leaves the group, and Group Replication is stopped on every member by a 
STOP GROUP_REPLICATION statement or system shutdown, the group now only exists in theory, as a configuration on the members. 
In that situation, to re-create the group, it must be started by bootstrapping as if it was being started for the first time.


collect information about the biggest transaction among all servers 
-----------------------------------------------------------------------------------

mysql>  SELECT @@HOSTNAME , @@GLOBAL.GTID_EXECUTED;


Use the member that has the biggest transaction set to bootstrap the group ( bosprod03 )
---------------------------------------------------------------------------------------------

mysql> SET GLOBAL group_replication_bootstrap_group=ON;
mysql> START GROUP_REPLICATION;
mysql> SET GLOBAL group_replication_bootstrap_group=OFF;

Add each of the other members back into the group, in any order, by issuing a START GROUP_REPLICATION 
----------------------------------------------------------------------------------------------------------

mysql> START GROUP_REPLICATION;


Changing primary in group replication
--------------------------------------------------

mysql> SELECT group_replication_set_as_primary('e5406206-28e9-11ef-b28f-000c293ea0bc');

mysql> select * from performance_schema.replication_group_members;


Changing mode to multi-primary from single-primary and vice versa in group replication
---------------------------------------------------------------------------------------------

mysql> SELECT group_replication_switch_to_multi_primary_mode();

mysql> select * from performance_schema.replication_group_members;
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------+
| CHANNEL_NAME              | MEMBER_ID                            | MEMBER_HOST | MEMBER_PORT | MEMBER_STATE | MEMBER_ROLE |
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------+
| group_replication_applier | 8da6031a-28e9-11ef-a875-000c29051378 | bosprod01   |        3306 | ONLINE       | PRIMARY     |
| group_replication_applier | e5406206-28e9-11ef-b28f-000c293ea0bc | bosprod02   |        3306 | ONLINE       | PRIMARY     |
| group_replication_applier | e89f4b84-28e9-11ef-bbb7-000c29009988 | bosprod03   |        3306 | ONLINE       | PRIMARY     |
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------+

mysql> SELECT group_replication_switch_to_single_primary_mode();

mysql> select * from performance_schema.replication_group_members;
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------
| CHANNEL_NAME              | MEMBER_ID                            | MEMBER_HOST | MEMBER_PORT | MEMBER_STATE | MEMBER_ROLE 
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------
| group_replication_applier | 8da6031a-28e9-11ef-a875-000c29051378 | bosprod01   |        3306 | ONLINE       | PRIMARY     
| group_replication_applier | e5406206-28e9-11ef-b28f-000c293ea0bc | bosprod02   |        3306 | ONLINE       | SECONDARY   
| group_replication_applier | e89f4b84-28e9-11ef-bbb7-000c29009988 | bosprod03   |        3306 | ONLINE       | SECONDARY   
+---------------------------+--------------------------------------+-------------+-------------+--------------+-------------


Group Replication Offline Upgrade 
-------------------------------------------------------------------

To perform offline upgrade , remove each member from the group and shutdown all the members, 
perform an upgrade and restart the group.

In multi-primary mode , group members can be shutdown in any order.

In single-primary mode , secondary should be shutdown and then finally primary.


Group Replication Online Upgrade 
--------------------------------------------------------------------

To perform online upgrade , remove a member from the group , shutdown and upgrade it and then rejoin it to the group.
so when upgrading one instance other 2 instances are available.

while upgrading single-primary group it is recommended to first upgrade all the secondaries and then the primary.

In [ ]:
Mysql Innodb Cluster 
=================================

Innodb cluster provides high availability solution for Mysql.

It uses 3 features to acheive this - Mysql shell , Group replication and Mysql Router.


Install Mysql shell in all servers
--------------------------------------------------------

[root@bosprod01 ~]# yum install mysql-shell

[root@bosprod01 ~]# mysqlsh

MySQL  localhost  SQL > \js
Switching to JavaScript mode...

MySQL  localhost  JS > \s

MySQL Shell version 8.4.0
Connection Id:                17
Current schema:               
Current user:                 root@localhost



Pre-Checking Instance Configuration for InnoDB Cluster Usage
----------------------------------------------------------------------------

MySQL  localhost  JS > dba.checkInstanceConfiguration('localhost');

ERROR: New account(s) with proper source address specification to allow remote connection from all
instances must be created to manage the cluster.


New admin account should be created in all instances to manage the cluster 
-----------------------------------------------------------------------------------

MySQL  localhost  JS > dba.configureInstance();

Configuring local MySQL instance listening at port 3306 for use in an InnoDB Cluster...

1) Create remotely usable account for 'root' with same grants and password
2) Create a new admin account for InnoDB Cluster with minimal required grants
3) Ignore and continue
4) Cancel

Please select an option [1]: 2

Account Name: icadmin
Password for new account: ************
Confirm password: ************


Check config again using new account created 
----------------------------------------------------------------

 MySQL  localhost  JS > dba.checkInstanceConfiguration('icadmin@localhost');
Please provide the password for 'icadmin@localhost': ************

NOTE: Some configuration options need to be fixed:
+----------------------------------------+---------------+----------------+--------------------------------------------------+
| Variable                               | Current Value | Required Value | Note                                             |
+----------------------------------------+---------------+----------------+--------------------------------------------------+
| binlog_transaction_dependency_tracking | COMMIT_ORDER  | WRITESET       | Update the server variable                       |
| enforce_gtid_consistency               | OFF           | ON             | Update read-only variable and restart the server |
| gtid_mode                              | OFF           | ON             | Update read-only variable and restart the server |
| server_id                              | 1             | <unique ID>    | Update read-only variable and restart the server |
+----------------------------------------+---------------+----------------+--------------------------------------------------+

Do you want to perform the required configuration changes? [y/n]: n


Modify the config file my.cnf with required changes 
-------------------------------------------------------------

[root@bosprod01 ~]# cat /etc/my.cnf
binlog-transaction-dependency-tracking=WRITESET
enforce-gtid-consistency=ON
gtid-mode=ON
server-id=1

[root@bosprod01 ~]# systemctl restart mysqld
[root@bosprod01 ~]# 


Perform  the same above steps in other two instances 
---------------------------------------------------------------



Creating an Innodb cluster 
-----------------------------------------

Once instances are prepared , create cluster using function dba.CreateCluster().

The instance where you are running this function becomes the seed and other instances becomes the replica which you add to
this cluster later.

MySQL  localhost  JS > \c icadmin@localhost

MySQL  localhost:33060+ ssl  JS > var cluster = dba.createCluster('bosprodcluster');

Cluster successfully created. Use Cluster.addInstance() to add MySQL instances.
At least 3 instances are needed for the cluster to be able to withstand up to
one server failure.


MySQL  localhost:33060+ ssl  JS > cluster.status();


Adding instances to an innodb cluster 
-------------------------------------------------------

MySQL  localhost:33060+ ssl  JS > cluster.addInstance('icadmin@bosprod02:3306');

Please select a recovery method [C]lone/[I]ncremental recovery/[A]bort (default Clone): C

NOTE: bosprod02:3306 is being cloned from bosprod01:3306
** Stage DROP DATA: Completed 
** Clone Transfer  
    FILE COPY  ############################################################  100%  Completed
    PAGE COPY  ############################################################  100%  Completed
    REDO COPY  ############################################################  100%  Completed

The instance 'bosprod02:3306' was successfully added to the cluster.



Cluster commands for monitoring inndodb clsuter 
--------------------------------------------------------------------

MySQL  localhost:33060+ ssl  JS > cluster = dba.getCluster();
<Cluster:bosproddbcluster>

MySQL  localhost:33060+ ssl  JS > cluster.status();

MySQL  localhost:33060+ ssl  JS > cluster.describe();

MySQL  localhost:33060+ ssl  JS > cluster.help();


Switchover primary
----------------------------------------------------------------------

mysql> select CHANNEL_NAME,MEMBER_HOST,MEMBER_STATE,MEMBER_ROLE
    -> from performance_schema.replication_group_members;

MySQL  bosprod02:33060+ ssl  JS > cluster.setPrimaryInstance("bosprod01");

Setting instance 'bosprod01' as the primary instance of cluster 'bosproddbcluster'...

Instance 'bosprod03:3306' remains SECONDARY.
Instance 'bosprod01:3306' was switched from SECONDARY to PRIMARY.
Instance 'bosprod02:3306' was switched from PRIMARY to SECONDARY.

The instance 'bosprod01' was successfully elected as primary.


Switching cluster to multi-primary mode and vice-versa 
-----------------------------------------------------------------------

MySQL  bosprod03:33060+ ssl  JS > cluster.switchToMultiPrimaryMode();

Switching cluster 'bosproddbcluster' to Multi-Primary mode...

Instance 'bosprod03:3306' remains PRIMARY.
Instance 'bosprod01:3306' was switched from SECONDARY to PRIMARY.
Instance 'bosprod02:3306' was switched from SECONDARY to PRIMARY.

The cluster successfully switched to Multi-Primary mode.

MySQL  bosprod03:33060+ ssl  JS >

+---------------------------+-------------+--------------+-------------+
| CHANNEL_NAME              | MEMBER_HOST | MEMBER_STATE | MEMBER_ROLE |
+---------------------------+-------------+--------------+-------------+
| group_replication_applier | bosprod03   | ONLINE       | PRIMARY     |
| group_replication_applier | bosprod01   | ONLINE       | PRIMARY     |
| group_replication_applier | bosprod02   | ONLINE       | PRIMARY     |
+---------------------------+-------------+--------------+-------------+


MySQL  bosprod03:33060+ ssl  JS > cluster.switchToSinglePrimaryMode();

Switching cluster 'bosproddbcluster' to Single-Primary mode...

Instance 'bosprod03:3306' remains PRIMARY.
Instance 'bosprod01:3306' was switched from PRIMARY to SECONDARY.
Instance 'bosprod02:3306' was switched from PRIMARY to SECONDARY.

WARNING: Existing connections that expected a R/W connection must be disconnected, 
i.e. instances that became SECONDARY.

The cluster successfully switched to Single-Primary mode.

MySQL  bosprod03:33060+ ssl  JS > 


+---------------------------+-------------+--------------+-------------+
| CHANNEL_NAME              | MEMBER_HOST | MEMBER_STATE | MEMBER_ROLE |
+---------------------------+-------------+--------------+-------------+
| group_replication_applier | bosprod03   | ONLINE       | PRIMARY     |
| group_replication_applier | bosprod01   | ONLINE       | SECONDARY   |
| group_replication_applier | bosprod02   | ONLINE       | SECONDARY   |
+---------------------------+-------------+--------------+-------------+


Rebooting a cluster from a major outage 
---------------------------------------------------------------

if a cluster experiences a complete outage we can configure it using 
dba.rebootClutserFromCompleteOutage().

A complete outage means that group replication has stopped on all member instances.


MySQL  localhost  JS > cluster = dba.getCluster();
Dba.getCluster: This function is not available through a session to a standalone instance 
(metadata exists, instance belongs to that metadata, but GR is not active) (MYSQLSH 51314)
 MySQL  localhost  JS > 


MySQL  bosprod01:33060+ ssl  JS > dba.rebootClusterFromCompleteOutage();
 
Restoring the Cluster 'bosproddbcluster' from complete outage...

The instance metadata for 'bosprod01:3306' was successfully updated.

The instance 'bosprod03:3306' was successfully rejoined to the cluster.

The instance 'bosprod02:3306' was successfully rejoined to the cluster.

The Cluster was successfully rebooted.

In [ ]:
Install Mysql Router 
=======================================

MySQL Router is part of InnoDB Cluster and is lightweight middleware that provides transparent 
routing between application and back-end MySQL Servers. 

It is used for a wide variety of use cases, such as providing high availability and scalability 
by routing database traffic to appropriate back-end MySQL servers.

Ideally it should be installed in an app server , for now we will install it in bosprod03


install mysql router
----------------------------------

[root@bosprod03 ~]# yum install mysql-router-community

Installed:
  mysql-router-community-8.4.0-1.el8.x86_64                                                            

Complete!
[root@bosprod03 ~]# 


bootstrap the mysqlrouter 
------------------------------------

[root@bosprod03 ~]# mysqlrouter --bootstrap icadmin@bosprod01:3306 --directory mysqlrouter --user=root
Please enter MySQL password for icadmin: 

# MySQL Router configured for the InnoDB Cluster 'bosproddbcluster'

After this MySQL Router has been started with the generated configuration

    $ mysqlrouter -c /root/mysqlrouter/mysqlrouter.conf

InnoDB Cluster 'bosproddbcluster' can be reached by connecting to:

## MySQL Classic protocol

- Read/Write Connections: localhost:6446
- Read/Only Connections:  localhost:6447
- Read/Write Split Connections: localhost:6450

## MySQL X protocol

- Read/Write Connections: localhost:6448
- Read/Only Connections:  localhost:6449

[root@bosprod03 ~]# 


start the mysqlrouter 
----------------------------------

[root@bosprod03 ~]# /root/mysqlrouter/start.sh 
[root@bosprod03 ~]# PID 4017 written to '/root/mysqlrouter/mysqlrouter.pid'
stopping to log to the console. Continuing to log to filelog

[root@bosprod03 ~]# 

[root@bosprod03 ~]# netstat -anlp | grep mysqlrouter


check read/write and read-only connection 
----------------------------------------------

+---------------------------+-------------+--------------+-------------+
| CHANNEL_NAME              | MEMBER_HOST | MEMBER_STATE | MEMBER_ROLE |
+---------------------------+-------------+--------------+-------------+
| group_replication_applier | bosprod03   | ONLINE       | SECONDARY   |
| group_replication_applier | bosprod01   | ONLINE       | PRIMARY     |
| group_replication_applier | bosprod02   | ONLINE       | SECONDARY   |
+---------------------------+-------------+--------------+-------------+


[root@bosprod02 ~]# mysql -h bosprod03 -P 6446 -u icadmin -p
Enter password: 

mysql> select @@hostname;
+------------+
| @@hostname |
+------------+
| bosprod01  |
+------------+

[root@bosprod02 ~]# mysql -h bosprod03 -P 6447 -u icadmin -p
Enter password: 

mysql> select @@hostname;
+------------+
| @@hostname |
+------------+
| bosprod03  |
+------------+